# MODULE 22 - BIG DATA ANALYSIS WITH SPARK

Step 0 - Environment Setup


In [21]:
import sys
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install colorama

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [22]:
#
# Step 0 - Environment Setup
#
!pip install pyspark

import sys
from   colorama import Fore
import datetime

def logStep(msg):
    print(Fore.WHITE + str(datetime.datetime.now()) + ' ' + Fore.YELLOW + msg + Fore.WHITE)
    sys.stdout.flush()

start_time = datetime.datetime.now()
logStep('STEP 0 - ENVIRONMENT PREPARATION======================')

import warnings
import findspark
from   pyspark     import SparkFiles
from   pyspark.sql import SparkSession

warnings.filterwarnings('ignore')

print(Fore.GREEN)
print(F'Copyright        : {sys.copyright}')
print(F'OS Platform      : {sys.platform}')
print(F'OS Name          : {os.name}')
print(F'OS HOME          : {os.environ.get("HOME")}')
print(F'OS uName         : {os.uname().sysname}')
print(F'OS NodeName      : {os.uname().nodename}')
print(F'OS Release       : {os.uname().release}')
print(F'OS Release Ver   : {os.uname().version}')
print(F'OS Machine       : {os.uname().machine}')
print(F'Process ID       : {os.getpid()}')
print(F'Parent Process   : {os.getppid()}')
#print(F'OS User          : {os.getlogin()}')
#print(F'OS User ID       : {os.getuid()}')
#print(F'OS Group ID      : {os.getgid()}')
#print(F'OS Effective ID  : {os.geteuid()}')
#print(F'OS Effective GID : {os.getegid()}')
print(F'Current dir      : {os.getcwd()}')
print(F'Python version   : {sys.version}')
print(F'Version info     : {sys.version_info}')
print(F'Python API Ver   : {sys.api_version}')
print(F'Executable       : {sys.executable}')
print(F'Hadoop home      : {os.environ.get("HADOOP_HOME")}')
print(F'Spark version    : {findspark.__version__}')
print(F'Spark home(Find) : {findspark.find()}')
print(F'Spark Home(Env)  : {os.environ.get("SPARK_HOME")}')
print(F'Spark UI         : http://localhost:4040')
print(F'Spark submit     : {sys.argv[0]}')
print(F'Java home        : {os.environ.get("JAVA_HOME")}')
print()

logStep("STEP 0 - DONE=========================================");
end_time           = datetime.datetime.now()
step0_elapsed_time = end_time - start_time
logStep(F"STEP 0 - ELAPSED TIME: {step0_elapsed_time} seconds=========")

2023-09-04 22:46:59.549578 STEP 0 - ENVIRONMENT PREPARATION======================

Copyright        : Copyright (c) 2001-2023 Python Software Foundation.
All Rights Reserved.

Copyright (c) 2000 BeOpen.com.
All Rights Reserved.

Copyright (c) 1995-2001 Corporation for National Research Initiatives.
All Rights Reserved.

Copyright (c) 1991-1995 Stichting Mathematisch Centrum, Amsterdam.
All Rights Reserved.
OS Platform      : linux
OS Name          : posix
OS HOME          : /root
OS uName         : Linux
OS NodeName      : 98e8a7515dda
OS Release       : 5.15.109+
OS Release Ver   : #1 SMP Fri Jun 9 10:57:30 UTC 2023
OS Machine       : x86_64
Process ID       : 3451
Parent Process   : 79
Current dir      : /content
Python version   : 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Version info     : sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)
Python API Ver   : 1013
Executable       : /usr/bin/python3
Hadoop home      : None
Spark version    : 2.0.1

Step 1 - Read in the source file into a DataFrame

In [23]:
#
# Step 1 - Read in the source file into a DataFrame.
#

start_time = datetime.datetime.now()
logStep("STEP 1 - READ SOURCE DATA=============================");
print(Fore.GREEN)
findspark.init()
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
home_df.show()

logStep("STEP 1 - DONE=========================================");
end_time           = datetime.datetime.now()
step1_elapsed_time = end_time - start_time
logStep(F"STEP 1 - ELAPSED TIME: {step1_elapsed_time} seconds=========")

2023-09-04 22:46:59.586099 STEP 1 - READ SOURCE DATA=============================

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   

Step 2 - Create a temporary view of the DataFrame.

In [24]:
#
# Step 2 - Create a temporary view of the DataFrame.
#

start_time         = datetime.datetime.now()
logStep("STEP 2 - CREATE A TEMPORARY VIEW======================")
print(Fore.GREEN)

home_df.createOrReplaceTempView('home_sales')

logStep("STEP 2 - DONE=========================================")
end_time           = datetime.datetime.now()
step2_elapsed_time = end_time - start_time
logStep(F"STEP 2 - ELAPSED TIME: {step2_elapsed_time} seconds=========")

2023-09-04 22:47:00.715101 STEP 2 - CREATE A TEMPORARY VIEW======================

2023-09-04 22:47:00.767316 STEP 2 - DONE=========================================
2023-09-04 22:47:00.770019 STEP 2 - ELAPSED TIME: 0:00:00.054797 seconds=========


Step 3 - What is the average price for a four bedroom house sold in each year rounded to two decimal places?


In [25]:
#
# Step 3 - What is the average price for a four bedroom house sold in each year rounded to two decimal places?
#

start_time         = datetime.datetime.now()
logStep("STEP 3 - WHAT IS THE AVERAGE PRICE?=4BR===============")
print(Fore.GREEN)

spark.sql("SELECT YEAR(date)          as YEAR, \
                  ROUND(AVG(price),2) as AVERAGE \
             FROM home_sales WHERE bedrooms == 4 \
                  GROUP BY YEAR(date) \
                  ORDER BY YEAR(date) desc").show()

logStep("STEP 3 - DONE=========================================")
end_time           = datetime.datetime.now()
step3_elapsed_time = end_time - start_time
logStep(F"STEP 3 - ELAPSED TIME: {step3_elapsed_time} seconds=========")

2023-09-04 22:47:00.779144 STEP 3 - WHAT IS THE AVERAGE PRICE?=4BR===============

+----+---------+
|YEAR|  AVERAGE|
+----+---------+
|2022|296363.88|
|2021|301819.44|
|2020|298353.78|
|2019| 300263.7|
+----+---------+

2023-09-04 22:47:02.015393 STEP 3 - DONE=========================================
2023-09-04 22:47:02.020687 STEP 3 - ELAPSED TIME: 0:00:01.241460 seconds=========


Set 4 - What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

In [26]:
#
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
#

start_time         = datetime.datetime.now()
logStep("STEP 4 - WHAT IS THE AVERAGE PRICE?=3BR/3B============")
print(Fore.GREEN)

spark.sql("SELECT date_built as YEAR , \
                  ROUND(AVG(price),2) as AVERAGE  \
             FROM home_sales WHERE bedrooms == 3 AND \
                                   bathrooms == 3 \
                  GROUP BY date_built \
                  ORDER BY date_built desc").show()

end_time           = datetime.datetime.now()
logStep("STEP 4 - DONE=========================================")
step4_elapsed_time = end_time - start_time
logStep(F"STEP 4 - ELAPSED TIME: {step4_elapsed_time} seconds=========")


2023-09-04 22:47:02.032577 STEP 4 - WHAT IS THE AVERAGE PRICE?=3BR/3B============

+----+---------+
|YEAR|  AVERAGE|
+----+---------+
|2017|292676.79|
|2016|290555.07|
|2015| 288770.3|
|2014|290852.27|
|2013|295962.27|
|2012|293683.19|
|2011|291117.47|
|2010|292859.62|
+----+---------+

2023-09-04 22:47:03.329508 STEP 4 - DONE=========================================
2023-09-04 22:47:03.332763 STEP 4 - ELAPSED TIME: 0:00:01.296885 seconds=========


Step 5 - What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet rounded to two decimal places?

In [27]:
#
# Step 5 - What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
#

start_time         = datetime.datetime.now()
logStep("STEP 5 - WHAT IS THE AVERAGE PRICE?=3R/3/2============")
print(Fore.GREEN)

spark.sql("SELECT date_built as YEAR, \
           ROUND(AVG(price),2) as AVERAGE from home_sales \
           where bedrooms  == 3 AND \
                 bathrooms == 3 AND \
                    floors == 2 AND \
               sqft_living >= 2000 \
                 group by date_built \
                 order by date_built desc").show()

end_time           = datetime.datetime.now()
logStep("STEP 5 - DONE=========================================")
step5_elapsed_time = end_time - start_time
logStep(F"STEP 5 - ELAPSED TIME: {step5_elapsed_time} seconds=========")


2023-09-04 22:47:03.354486 STEP 5 - WHAT IS THE AVERAGE PRICE?=3R/3/2============

+----+---------+
|YEAR|  AVERAGE|
+----+---------+
|2017|280317.58|
|2016| 293965.1|
|2015|297609.97|
|2014|298264.72|
|2013|303676.79|
|2012|307539.97|
|2011|276553.81|
|2010|285010.22|
+----+---------+

2023-09-04 22:47:04.903917 STEP 5 - DONE=========================================
2023-09-04 22:47:04.909984 STEP 5 - ELAPSED TIME: 0:00:01.543062 seconds=========


Step 6 - What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000? Although this is a small dataset, determine the run time for this query.

In [28]:
#
# Ste 6 - What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000? Although this is a small dataset, determine the run time for this query.
#

start_time         = datetime.datetime.now()
logStep("STEP 6 - WHAT IS THE AVERAGE PRICE?=300K==============")
print(Fore.GREEN)

spark.sql("SELECT view as VIEW, \
                  ROUND(AVG(price),2) AS AVERAGE \
             FROM home_sales \
             GROUP BY view \
           HAVING ROUND(AVG(price),2) >= 350000 \
           ORDER BY view").show()

end_time           = datetime.datetime.now()
logStep("STEP 6 - DONE=========================================")
step6_elapsed_time = end_time - start_time
logStep(F"STEP 6 - ELAPSED TIME: {step6_elapsed_time} seconds=========")


2023-09-04 22:47:04.946580 STEP 6 - WHAT IS THE AVERAGE PRICE?=300K==============

+----+---------+
|VIEW|  AVERAGE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

2023-09-04 22:47:06.380453 STEP 6 - DONE=========================================
2023-09-04 22:47:06.383404 STEP 6 - ELAPSED TIME: 0:00:01.433867 seconds=========


Step 7 - Cache the the temporary table home_sales.

In [29]:
#
# Step 7 - Cache the the temporary table home_sales.
#

start_time         = datetime.datetime.now()
logStep("STEP 7 - CACHE HOME DATA==============================")
print(Fore.GREEN)

spark.sql("cache table home_sales")

end_time           = datetime.datetime.now()
logStep("STEP 7 - DONE=========================================")
step7_elapsed_time = end_time - start_time
logStep(F"STEP 7 - ELAPSED TIME: {step7_elapsed_time} seconds=========")

2023-09-04 22:47:06.404413 STEP 7 - CACHE HOME DATA==============================

2023-09-04 22:47:08.833319 STEP 7 - DONE=========================================
2023-09-04 22:47:08.835002 STEP 7 - ELAPSED TIME: 0:00:02.428916 seconds=========


Step 8 - Check if the table is cached.

In [30]:
#
# Step 8 - Check if the table is cached.
#

start_time         = datetime.datetime.now()
logStep("STEP 8 - IS THE DATA CACHED===========================")
print(Fore.GREEN)

if (spark.catalog.isCached('home_sales') == False):
    print("home_sales is not cached")
else:
    print("home_sales is cached")
print(Fore.GREEN)
end_time           = datetime.datetime.now()
logStep("STEP 8 - DONE=========================================")
step8_elapsed_time = end_time - start_time
logStep(F"STEP 8 - ELAPSED TIME: {step8_elapsed_time} seconds=========")

2023-09-04 22:47:08.845558 STEP 8 - IS THE DATA CACHED===========================

home_sales is cached

2023-09-04 22:47:08.868581 STEP 8 - DONE=========================================
2023-09-04 22:47:08.876496 STEP 8 - ELAPSED TIME: 0:00:00.023028 seconds=========


Step 9 - Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

In [31]:
#
# Step 9 - Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
#

start_time         = datetime.datetime.now()
logStep("STEP 9 - REPEAT QUERY=================================")
print(Fore.GREEN)

spark.sql("SELECT view as VIEW, \
                  ROUND(AVG(price),2) AS AVERAGE \
             FROM home_sales \
             GROUP BY view \
           HAVING ROUND(AVG(price),2) >= 350000 \
           ORDER BY view").show()

end_time           = datetime.datetime.now()
logStep("STEP 9 - DONE=========================================")
step9_elapsed_time = end_time - start_time
logStep(F"STEP 9 - ELAPSED TIME: {step9_elapsed_time} seconds=========")


2023-09-04 22:47:08.892363 STEP 9 - REPEAT QUERY=================================

+----+---------+
|VIEW|  AVERAGE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

2023-09-04 22:47:09.552206 STEP 9 - DONE=========================================
2023-09-04 22:47:09.556962 STEP 9 - ELAPSED TIME: 0:00:00.659833 seconds=========


Step 9.1 - Determine the runtime and compare to the original runtime

In [32]:
#
# Step 9.1 Determine the runtime and compare to the original runtime
#

start_time         = datetime.datetime.now()
logStep("STEP 9.1 - RUNTIME DIFFERENCE =========================")
print(Fore.GREEN)

print(F"Time required for a non-cached Query: {step6_elapsed_time}")
print(F"Time required for a cached Query    : {step9_elapsed_time}")
time_difference = step6_elapsed_time - step9_elapsed_time
print(F"Time difference                     : {time_difference}")
end_time           = datetime.datetime.now()
print(Fore.GREEN)

logStep("STEP 9.1 - DONE========================================")
step91_elapsed_time = end_time - start_time
logStep(F"STEP 9.1 - ELAPSED TIME: {step91_elapsed_time} seconds========")

2023-09-04 22:47:09.569638 STEP 9.1 - RUNTIME DIFFERENCE =========================

Time required for a non-cached Query: 0:00:01.433867
Time required for a cached Query    : 0:00:00.659833
Time difference                     : 0:00:00.774034

2023-09-04 22:47:09.576140 STEP 9.1 - DONE========================================
2023-09-04 22:47:09.578936 STEP 9.1 - ELAPSED TIME: 0:00:00.004246 seconds========


Step 10 - Partition by the "date_built" field on the formatted parquet home sales data


In [33]:
#
# Step 10 - Partition by the "date_built" field on the formatted parquet home sales data
#

start_time         = datetime.datetime.now()
logStep("STEP 10 - FORMATTED PARQUET============================")
print(Fore.GREEN)
home_df.write.parquet('home_parquet', mode='overwrite',partitionBy='date_built')
home_df.show()


end_time           = datetime.datetime.now()
logStep("STEP 10 - DONE=========================================")
step10_elapsed_time = end_time - start_time
logStep(F"STEP 10 - ELAPSED TIME: {step10_elapsed_time} seconds=========")

2023-09-04 22:47:09.591717 STEP 10 - FORMATTED PARQUET============================

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|  

In [34]:
#
# Step 11 - Read the parquet formatted data.
#

start_time         = datetime.datetime.now()
logStep("STEP 11 - READ PARQUET=================================")
print(Fore.GREEN)

parquet_home_df = spark.read.parquet('home_parquet')
parquet_home_df.show()

end_time           = datetime.datetime.now()
logStep("STEP 11 - DONE=========================================")
step11_elapsed_time = end_time - start_time
logStep(F"STEP 11 - ELAPSED TIME: {step11_elapsed_time} seconds=========")

2023-09-04 22:47:12.235102 STEP 11 - READ PARQUET=================================

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      20

Step 12 - Create a temporary table for the parquet data.

In [35]:
#
# Step 12 - Create a temporary table for the parquet data.
#

start_time         = datetime.datetime.now()
logStep("STEP 12 - CREATE PARQUET VIEW==========================")
print(Fore.GREEN)

parquet_home_df.createOrReplaceTempView('parquet_temp_home')
parquet_home_df.show()

end_time           = datetime.datetime.now()
logStep("STEP 12 - DONE=========================================")
step12_elapsed_time = end_time - start_time
logStep(F"STEP 12 - ELAPSED TIME: {step12_elapsed_time} seconds=========")

2023-09-04 22:47:12.930760 STEP 12 - CREATE PARQUET VIEW==========================

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      20

Step 13 - Run the query that filters out the view ratings with average price of greater than or equal to $350,000 with the parquet DataFrame. Round your average to two decimal places

In [36]:
#
# Step 13 - Run the query that filters out the view ratings with average price of greater than or equal to $350,000 with the parquet DataFrame. Round your average to two decimal places.
#

start_time         = datetime.datetime.now()
logStep("STEP 13 - REPEAT QUERY=================================")
print(Fore.GREEN)

spark.sql("SELECT view as VIEW, \
                  ROUND(AVG(price),2) AS AVERAGE \
             FROM parquet_temp_home \
             GROUP BY view \
           HAVING ROUND(AVG(price),2) >= 350000 \
           ORDER BY view").show()

end_time           = datetime.datetime.now()
logStep("STEP 13 - DONE=========================================")
step13_elapsed_time = end_time - start_time
logStep(F"STEP 13 - ELAPSED TIME: {step13_elapsed_time} seconds=========")

2023-09-04 22:47:13.157767 STEP 13 - REPEAT QUERY=================================

+----+---------+
|VIEW|  AVERAGE|
+----+---------+
| 100|1026669.5|
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
+----+---------+
only showing top 20 rows

2023-09-04 22:47:13.722215 STEP 13 - DONE=========================================
2023-09-04 22:47:13.725371 STEP 13 - ELAPSED TIME: 0:00:00.564421 seconds=========


In [37]:
#
# Step 13.1 - Determine the runtime and compare it to the cached version.
#

start_time         = datetime.datetime.now()
logStep("STEP 13.1 - RUNTIME DIFFERENCE ========================")
print(Fore.GREEN)
print(F"Time required for a non-cached Query    : {step6_elapsed_time}")
print(F"Time required for a parquet cached Query: {step13_elapsed_time}")
time_difference = step6_elapsed_time - step13_elapsed_time
print(F"Time difference                         : {time_difference}")
print(Fore.GREEN)
end_time           = datetime.datetime.now()
logStep("STEP 13.1 - DONE=======================================")
step131_elapsed_time = end_time - start_time
logStep(F"STEP 13.1 - ELAPSED TIME: {step131_elapsed_time} seconds=======")

2023-09-04 22:47:13.736739 STEP 13.1 - RUNTIME DIFFERENCE ========================

Time required for a non-cached Query    : 0:00:01.433867
Time required for a parquet cached Query: 0:00:00.564421
Time difference                         : 0:00:00.869446

2023-09-04 22:47:13.740055 STEP 13.1 - DONE=======================================
2023-09-04 22:47:13.741566 STEP 13.1 - ELAPSED TIME: 0:00:00.003354 seconds=======


In [38]:
#
# 14. Un-cache the home_sales temporary table.
#

start_time         = datetime.datetime.now()
logStep("STEP 14 - UN-CACHE=====================================")
print(Fore.GREEN)

spark.sql("uncache table home_sales")

end_time           = datetime.datetime.now()
logStep("STEP 14 - DONE=========================================")
step14_elapsed_time = end_time - start_time
logStep(F"STEP 14 - ELAPSED TIME: {step14_elapsed_time} seconds=========")

2023-09-04 22:47:13.753919 STEP 14 - UN-CACHE=====================================

2023-09-04 22:47:13.780680 STEP 14 - DONE=========================================
2023-09-04 22:47:13.782286 STEP 14 - ELAPSED TIME: 0:00:00.026760 seconds=========


In [39]:
#
# Step 15. Check if the home_sales is no longer cached
#
start_time         = datetime.datetime.now()
logStep("STEP 15 - CACHE= CHECK=================================")
print(Fore.GREEN)

if (spark.catalog.isCached('home_sales') == False):
    print("home_sales is not cached")
else:
    print("home_sales is cached")
print(Fore.GREEN)

end_time           = datetime.datetime.now()
logStep("STEP 15 - DONE=========================================")
step15_elapsed_time = end_time - start_time
logStep(F"STEP 15 - ELAPSED TIME: {step15_elapsed_time} seconds=========")


2023-09-04 22:47:13.793801 STEP 15 - CACHE= CHECK=================================

home_sales is not cached

2023-09-04 22:47:13.808059 STEP 15 - DONE=========================================
2023-09-04 22:47:13.811154 STEP 15 - ELAPSED TIME: 0:00:00.014259 seconds=========


In [40]:
logStep("=========TOTAL RUN TIME================================")
print(Fore.GREEN)
print('Total Runtime, per Step')
print('')
print(F"Step  0:   {step0_elapsed_time} seconds")
print(F"Step  1:   {step1_elapsed_time} seconds")
print(F"Step  2:   {step2_elapsed_time} seconds")
print(F"Step  3:   {step3_elapsed_time} seconds")
print(F"Step  4:   {step4_elapsed_time} seconds")
print(F"Step  5:   {step5_elapsed_time} seconds")
print(F"{Fore.RED}Step  6:   {step6_elapsed_time} seconds - non-cached query{Fore.GREEN}")
print(F"{Fore.GREEN}Step  7:   {step7_elapsed_time} seconds")
print(F"Step  8:   {step8_elapsed_time} seconds")
print(F"{Fore.RED}Step  9:   {step9_elapsed_time} seconds - cached query")
print(F"{Fore.GREEN}Step  9.1: {step91_elapsed_time} seconds")
print(F"Step 10:   {step10_elapsed_time} seconds")
print(F"Step 11:   {step11_elapsed_time} seconds")
print(F"Step 12:   {step12_elapsed_time} seconds")
print(F"{Fore.RED}Step 13:   {step13_elapsed_time} seconds - parquet cached query")
print(F"{Fore.GREEN}Step 13.1: {step131_elapsed_time} seconds")
print(F"Step 14:   {step14_elapsed_time} seconds")
print(F"Step 15:   {step15_elapsed_time} seconds")
print(F"{Fore.YELLOW}Total  :   {step0_elapsed_time + step1_elapsed_time + step2_elapsed_time + step3_elapsed_time + step4_elapsed_time + step5_elapsed_time + step6_elapsed_time + step7_elapsed_time + step8_elapsed_time + step9_elapsed_time + step91_elapsed_time + step10_elapsed_time + step11_elapsed_time + step12_elapsed_time + step13_elapsed_time + step131_elapsed_time + step14_elapsed_time + step15_elapsed_time} seconds")
print('')
print(Fore.GREEN + 'Runtime Reduction')
print('')
print(F'{Fore.RED}Cached Query Reduction  : {100-(step9_elapsed_time/step6_elapsed_time*100):.2f}%')
print(F'Parquet Query Reduction : {100-(step13_elapsed_time/step6_elapsed_time*100):.2f}%')
print('')
logStep("=========END============================================")

2023-09-04 22:47:13.828006 =========TOTAL RUN TIME================================

Total Runtime, per Step

Step  0:   0:00:00.022705 seconds
Step  1:   0:00:01.089260 seconds
Step  2:   0:00:00.054797 seconds
Step  3:   0:00:01.241460 seconds
Step  4:   0:00:01.296885 seconds
Step  5:   0:00:01.543062 seconds
Step  6:   0:00:01.433867 seconds - non-cached query
Step  7:   0:00:02.428916 seconds
Step  8:   0:00:00.023028 seconds
Step  9:   0:00:00.659833 seconds - cached query
Step  9.1: 0:00:00.004246 seconds
Step 10:   0:00:02.628984 seconds
Step 11:   0:00:00.680673 seconds
Step 12:   0:00:00.215039 seconds
Step 13:   0:00:00.564421 seconds - parquet cached query
Step 13.1: 0:00:00.003354 seconds
Step 14:   0:00:00.026760 seconds
Step 15:   0:00:00.014259 seconds
Total  :   0:00:13.931549 seconds

Runtime Reduction

Cached Query Reduction  : 53.98%
Parquet Query Reduction : 60.64%

2023-09-04 22:47:13.840328 =========END============================================
